致阅读者,

小鸟快跑是我在业余时间设计的一个单人跑团(桌上角色扮演游戏(TRPG))游戏, 计划使用GPT API来驱动剧情生成, 但为了在作品集中进行展示进行了一定简化。

我希望重点向您展示我在数值设计上的一些想法, 这包括下面我设计的的TRPG伤害与状态数值计算系统:

## 数学公式定义

### 符号定义

- $A$ 代表小鸟 $a$ 的攻击力
- $C_a$ 代表小鸟 $a$ 的爪子锋利度
- $B_a$ 代表小鸟 $a$ 的鸟喙锋利度
- $D$ 代表小鸟 $b$ 的耐久值
- $E_a$ 代表小鸟 $a$ 的忍耐力
- $I_a$ 代表小鸟 $a$ 的威慑力
- $I_b$ 代表小鸟 $b$ 的威慑力
- $G_a$ 代表小鸟 $a$ 的敏捷度
- $f(I_a, I_b)$ 代表状态检测函数，考虑 $a$ 和 $b$ 的威慑力差异

### 状态检测函数 $f(I_a, I_b)$

状态检测函数由三部分组成：

1. $R_1 = \begin{cases} -0.5 & \text{if } E_a > 0.7 \\ 0 & \text{otherwise} \end{cases}$
2. $R_2 = \begin{cases} 1 & \text{if } |I_a - I_b| > 0.3 \\ 0 & \text{otherwise} \end{cases}$
3. $R_3 = \begin{cases} -0.5 & \text{if } 0 < I_b - I_a \leq 0.3 \\ 0 & \text{otherwise} \end{cases}$

则状态检测函数为：
$$f(I_a, I_b) = R_1 + R_2 + R_3$$

### 拟定伤害公式 $H$

拟定伤害的计算依赖于状态检测函数和是否闪避成功：

- 闪避成功的概率 $P = \frac{G_a}{6}$
- 若闪避成功，则 $H = 0$
- 若闪避失败，则 $$H = (A + C_a + B_a) \times (1 + f(I_a, I_b)) - D$$

综上，完整的拟定伤害公式可以用条件期望表示为：
$$H = \begin{cases} 0 & \text{with probability } P \\ (A + C_a + B_a) \times (1 + f(I_a, I_b)) - D & \text{with probability } 1 - P \end{cases}$$




In [ ]:
import ipywidgets as widgets
from IPython.display import display
import random

# 定义一个小鸟类，包含各种战斗属性
class Bird:
    def __init__(self, attack, durability, intimidation, endurance, luck, agility, claw_sharpness, beak_sharpness):
        self.attack = attack  # 攻击力
        self.durability = durability  # 耐久值
        self.intimidation = intimidation  # 威慑力
        self.endurance = endurance  # 忍耐力
        self.luck = luck  # 幸运值
        self.agility = agility  # 敏捷度
        self.claw_sharpness = claw_sharpness  # 爪子锋利度
        self.beak_sharpness = beak_sharpness  # 鸟喙锋利度

# 定义战役类，用于模拟两只小鸟的战斗
class Battle:
    def __init__(self, bird_a, bird_b):
        self.bird_a = bird_a  # 小鸟A
        self.bird_b = bird_b  # 小鸟B

    # 计算战斗中的伤害
    def calculate_damage(self):
        status_effect = self.status_check(self.bird_a, self.bird_b)
        damage = (self.bird_a.attack + self.bird_a.claw_sharpness + self.bird_a.beak_sharpness) * (1 + status_effect) - self.bird_b.durability
        return damage

    # 检查和计算状态影响
    def status_check(self, a, b):
        result1 = -0.5 if a.endurance > 0.7 else 0  # 根据忍耐力调整结果
        result2 = 1 if abs(a.intimidation - b.intimidation) > 0.3 else 0  # 根据威慑力差异调整结果
        result3 = -0.5 if 0 < (b.intimidation - a.intimidation) <= 0.3 else 0  # 根据微小的威慑力差异调整结果
        return result1 + result2 + result3

# 创建属性输入控件
attributes = ['attack', 'durability', 'intimidation', 'endurance', 'luck', 'agility', 'claw_sharpness', 'beak_sharpness']
bird1_inputs = {attr: widgets.FloatText(value=0, description=f'Bird1 {attr}:') for attr in attributes}
bird2_inputs = {attr: widgets.FloatText(value=0, description=f'Bird2 {attr}:') for attr in attributes}

# 创建交战和随机化按钮
button = widgets.Button(description="对战！")
randomize_button = widgets.Button(description="随机小鸟们的属性吧~")
output = widgets.Output()

# 定义战斗按钮的回调函数
def on_button_clicked(b):
    with output:
        output.clear_output()
        bird1 = Bird(**{attr: bird1_inputs[attr].value for attr in attributes})
        bird2 = Bird(**{attr: bird2_inputs[attr].value for attr in attributes})
        battle = Battle(bird1, bird2)
        damage = battle.calculate_damage()
        print(f"1号小鸟 攻击 2号小鸟 造成 {damage:.0f} 点伤害.")

# 定义随机化按钮的回调函数
def randomize_attributes(b):
    for bird_inputs in [bird1_inputs, bird2_inputs]:
        for attr in attributes:
            bird_inputs[attr].value = random.uniform(1, 10)  # 为每个属性分配1到10之间的随机值

button.on_click(on_button_clicked)
randomize_button.on_click(randomize_attributes)

# 显示输入控件和按钮
inputs = widgets.VBox([*bird1_inputs.values(), *bird2_inputs.values(), button, randomize_button])
display(inputs, output)


Output()

你可以在上面的属性栏中输入你期望的属性值，并展开模拟对战，也可以按下随机按钮来进行随机对战。

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

class Context:
    def __init__(self):
        self.story_blocks = [
            [
                "小鸟首次进入屋子，被玄关处整齐排列的鞋子和挂钩上整洁的外套所吸引。它好奇地窜进一个半开的鞋盒里探险。",
                "一天下班后，白领似乎心情不佳，随手将鞋子脱在玄关，没有按惯例整理。小鸟猜想她的情绪消沉。",
                "小鸟发现衣柜门轻微地开着，它飞进去探索那些挂着的长外衣和羊毛围巾，找到了一个温暖的角落打了个小盹。",
                "白领准备外出时，小鸟注意到她在挑选衣服时显得犹豫不决，她从衣柜中取出一件又一件衣服，然后叹了口气放回去。",
                "玄关处挂着一面小镜子，小鸟第一次看到自己的倒影，它好奇地扑腾翅膀，试图与倒影中的自己互动。"
            ],
            [
                "小鸟被客厅窗台上的雨滴声吸引，从窗台看出去，只见雨雾中偶尔闪着车灯，像鱼一样游过。",
                "在一个阴雨的下午，白领打开液晶电视，播放了一部老电影。小鸟被电视屏幕上变换的光影所吸引，跳到电视柜上仔细观看。",
                "小鸟注意到油画花架上摆放的画作，画中的景色与窗外的阴暗大相径庭。它走向那明亮而温暖的图景，假装自己要在其中飞翔。",
                "白领坐在窗边的沙发上，静静地望着窗外的雨景，车流让她的面孔一闪而过地照亮，小鸟感到她的心情似乎也跟着天气起伏。",
                "在一次清洁时，白领小心翼翼地擦拭油画和花架，小鸟看到她在每一次抚摸画框时，都似乎在回忆着什么，她的表情温柔而又略带忧伤。"
            ],
            [
                "小鸟跳上茶几，发现白领留下的一杯未喝完的茶。茶水中倒映出窗外的灰暗天空，小鸟感觉到了房间里和茶一样的寂静氛围。",
                "在一个休息日，白领坐在那张包豪斯风格的椅子上，翻看一本旧书。小鸟感受到她的专注和内心的宁静，与工作日的忙碌形成鲜明对比。",
                "小鸟探索客厅一角的纸盒和袋子，里面装满了购物的痕迹。白领偶尔的小购物似乎是她对抗单调生活的小确幸。",
                "白领和小鸟一起在茶几旁玩一个简单的投掷游戏，白领用纸团轻轻投向小鸟，两者都显示出一种简单但真挚的喜悦。",
                "白领在清理堆积的纸盒和袋子时，小鸟看到她略显疲惫但又满足的面容，似乎在清理物品的同时也在整理自己的思绪。"
            ],
            [
                "在昏暗的卧室里，小鸟在自己的鸟窝中观察着白领。她躺在单人床上，眼神空洞地望着天花板，似乎在思考些什么深沉的问题。",
                "小鸟注意到卧室的墙壁上没有任何装饰，只有那个鸟窝。墙壁与床单组成一个几何图形，映衬出白领内心的空旷。",
                "白领在卧室中与小鸟分享自己的梦境，她的声音低沉而有些颤抖，小鸟感到了她的不安和深藏的痛苦。",
                "某个清晨，小鸟从鸟窝里出来，看到白领已经起床整理床铺。她的动作机械而快速，好像在逃避什么。",
                "一次夜晚，白领无法入睡，她坐在床边低声哭泣。小鸟静静地看着，感到了前所未有的共鸣，它轻轻地飞到她旁边，试图给予安慰。"
            ],
            [
                "小鸟瞧着白领在一体式灶台上备菜，她的动作充满了热情和享受。尽管厨房小，每一个动作都显得有格外的意义。",
                "白领将自己的画作挂在厨房的一面墙上，色彩鲜明的画与厨房的白色瓷砖形成对比，小鸟觉得这些画作为厨房带来了生机。",
                "一次早餐，白领在高脚餐桌上摆满了自制的美食，小鸟坐在旁边的椅子上，感受到白领从烹饪中得到的快乐。",
                "白领打开袖珍小冰箱，里面整齐地排列着各种食材。小鸟注意到，虽然空间有限，但白领总能巧妙地使用每一寸空间。",
                "在一次深夜，白领不睡觉，而是选择在厨房里烘焙。小鸟闻着飘来的香气，感受到夜晚中这份独特的安宁与满足。"
            ],
            [
                "小鸟第一次偷偷跳进拉达旅行车时，白领一开始吃了一惊，但很快笑了起来，她的笑声充满了惊喜和温暖。",
                "每次出行，白领都会打开车窗让小鸟能感受到外面的风，她喜欢看小鸟享受微风拂过羽毛的样子。",
                "在停车场，白领和小鸟共享一份从市中心新鲜买回的食物。小鸟觉得，这辆车不仅是交通工具，更是他们共同的小天地。",
                "一次回家途中，白领开车穿过一片树林，她故意放慢速度，让小鸟能更好地看看外面的自然风光。",
                "在一个雨天，白领在车内播放轻音乐，而小鸟坐在副驾驶座上，静静地听着音乐和雨声，感觉非常舒适和安心。"
            ]
        ]

    def get_random_story_block(self, index):
        return random.choice(self.story_blocks[index])

# 创建实例
context = Context()

# 创建按钮
button = widgets.Button(description="投骰子前进吧")
output = widgets.Output()

# 定义按钮的响应函数
def on_button_clicked(b):
    with output:
        clear_output()
        # 随机生成一个 0-5 的整数
        dice_roll = random.randint(0, 5)
        # 获取并打印相应主题的随机剧情块
        print(dice_roll+1, "!")
        print(context.get_random_story_block(dice_roll))

# 将响应函数绑定到按钮上
button.on_click(on_button_clicked)

# 显示按钮和输出区域
display(button, output)


Button(description='投骰子前进吧', style=ButtonStyle())

Output()